In [1]:
load('packaging.rda', verbose=T)

Loading objects:
  save_functions
  set_parallel


### Data handling 

#### Restore factors **_data.frame_**

In [2]:
restore.factors <- function(x, reference, factor.vars=NULL){
    
    if (is.null(factor.vars)){
        factor.vars <- colnames(reference)[unlist(lapply(reference, is.factor))]
    }
    for (fct.var in factor.vars){
        if (!fct.var%in%colnames(x)){next()}
        if (!all(unique(x[,fct.var])%in%levels(reference[,fct.var]))){next()}
        x[,fct.var] <- factor(as.character(x[,fct.var]), levels=levels(reference[,fct.var]))
            
    }
    return(x)
}

#### Summarise by groups **_data.frame_**

In [3]:
summarise.metadata <- function(meta.data, group.vars, annotation.vars, 
                               keep_factors = T,
                               num.fun = mean, 
                               char.fun=function(x){paste(unique(as.character(x)), collapse=',')}){
    
    # copy data
    d <- meta.data
    
    require(dplyr)
    
    d %>% group_by_at(group.vars) %>% select_at(annotation.vars) %>% summarise_all(function(x){
        if (is.numeric(x)){num.fun(x)}else{char.fun(x)}
    }) %>% as.data.frame() -> sum.d
    
    if (keep_factors){
        sum.d <- restore.factors(x = sum.d, reference=meta.data)
    }
    # rownames(sum.d) <- apply(sum.d[,group.vars], 1, paste, collapse='_')
    return(sum.d)
    
}

In [4]:
save_functions('data_handling')

Loading required package: glue



[1] "restore.factors <- function (x, reference, factor.vars = NULL) \n{\n    if (is.null(factor.vars)) {\n        factor.vars <- colnames(reference)[unlist(lapply(reference, \n            is.factor))]\n    }\n    for (fct.var in factor.vars) {\n        if (!fct.var %in% colnames(x)) {\n            (next)()\n        }\n        if (!all(unique(x[, fct.var]) %in% levels(reference[, \n            fct.var]))) {\n            (next)()\n        }\n        x[, fct.var] <- factor(as.character(x[, fct.var]), levels = levels(reference[, \n            fct.var]))\n    }\n    return(x)\n}\n\nsave_functions <- function (basename = \"functions\", functions = NULL, envir.get = NULL, \n    R_script = T, Rda = T) \n{\n    require(glue)\n    if (is.null(envir.get)) {\n        envir.get <- globalenv()\n    }\n    if (is.null(functions)) {\n        env.vars <- sort(ls(envir = envir.get))\n        functions <- env.vars[sapply(sapply(env.vars, get, envir = envir.get), \n            class) == \"function\"]\n    }\n    if (Rda) {\n        save(list = functions, envir = envir.get, file = glue(\"{basename}.rda\"))\n    }\n    if (R_script) {\n        script <- paste(sapply(1:length(functions), function(i, \n            e = envir.get) {\n            paste(functions[i], paste(deparse(get(x = functions[i], \n                envir = e)), collapse = \"\\n\"), sep = \" <- \")\n        }), collapse = \"\\n\\n\")\n        cat(script, file = glue(\"{basename}.r\"))\n        return(script)\n    }\n}\n\nset_parallel <- function (slurm_check = T, max_cores = Inf, future = T, future.strategy = \"multicore\", \n    future.mem = Inf) \n{\n    n.cores <- NA\n    if (slurm_check) {\n        n.cores <- as.numeric(Sys.getenv(\"SLURM_CPUS_PER_TASK\"))\n    }\n    n.cores <- if (!is.na(n.cores) & n.cores > 1) \n        n.cores\n    else parallel::detectCores()\n    n.cores <- min(max_cores, n.cores)\n    if (future) {\n        if (!is.null(future.mem)) {\n            options(future.globals.maxSize = future.mem)\n        }\n        future::plan(strategy = future.strategy, workers = n.cores)\n    }\n    return(n.cores)\n}\n\nsummarise.metadata <- function (meta.data, group.vars, annotation.vars, keep_factors = T, \n    num.fun = mean, char.fun = function(x) {\n        paste(unique(as.character(x)), collapse = \",\")\n    }) \n{\n    d <- meta.data\n    require(dplyr)\n    sum.d <- d %>% group_by_at(group.vars) %>% select_at(annotation.vars) %>% \n        summarise_all(function(x) {\n            if (is.numeric(x)) {\n                num.fun(x)\n            }\n            else {\n                char.fun(x)\n            }\n        }) %>% as.data.frame()\n    if (keep_factors) {\n        sum.d <- restore.factors(x = sum.d, reference = meta.data)\n    }\n    return(sum.d)\n}"